# Running a simple control-loop

## After  completing this tutorial you will be able to invoke a simple control-loop which regulates a trafo-tap-changer

First we have to load all necesary packages. Of course we need the pandapower-package but also the control module and 
the network module.

In [1]:
# Importing necessary packages
import pandapower as pp
import control
from pandapower.networks import mv_oberrhein

We load the network and pass it to the parameter net. A first powerflow gives us results. We have a look at the trafo-resulttable. 

In [2]:
# loading the network with the usecase 'generation'
net = mv_oberrhein()
pp.runpp(net)

# checking the results
print(net.res_trafo)

       p_hv_mw  q_hv_mvar    p_lv_mw  q_lv_mvar     pl_mw   ql_mvar   i_hv_ka  \
114  17.270680   3.955948 -17.207125  -2.635336  0.063556  1.320612  0.092995   
142  20.863017   4.653035 -20.784893  -2.789431  0.078123  1.863605  0.112193   

      i_lv_ka  vm_hv_pu  va_hv_degree  vm_lv_pu  va_lv_degree  loading_percent  
114  0.495289       1.0           0.0  1.014598     -4.212528        70.871819  
142  0.588438       1.0           0.0  1.028804     -4.943953        85.502393  


To show off a simple control-behavior, we define a **ContiniousTapControl** for the tranformer with the ID 114 and pass the voltagesetpoint 0.98 p.u. with a small tolerance.

In [3]:
# initialising controller
tol = 1e-6
trafo_controller = control.ContinuousTapControl(net=net, tid=114, u_set=0.98, tol=tol)

We invoke the control-loop by setting **run_control=True** within the loadflow arguments.

In [4]:
# running a control-loop
pp.runpp(net, run_control=True)

In [5]:
# checking the results
print(net.res_trafo)

       p_hv_mw  q_hv_mvar    p_lv_mw  q_lv_mvar     pl_mw   ql_mvar   i_hv_ka  \
114  17.299985   4.185280 -17.235828  -2.762132  0.064157  1.423147  0.093421   
142  20.863017   4.653035 -20.784893  -2.789431  0.078123  1.863605  0.112193   

      i_lv_ka  vm_hv_pu  va_hv_degree  vm_lv_pu  va_lv_degree  loading_percent  
114  0.514188       1.0           0.0  0.980000     -4.513536        71.247940  
142  0.588438       1.0           0.0  1.028804     -4.943953        85.502393  


After comparing both resulttables it is clearly shown, that the voltage at the controlled transformer has changed from 1.014598 p.u. to the defined voltagesetpoint 0.98 p.u. 